# Extraction Script

After using the [ROI extracting script](./roi_extracting_script.py) for getting all the regions of interest (ROI) of the diagrams, we will run easyOCR to recognize the characters.

In order to avoid using too much computational power running easyOCR on each ROI, the ROIs will be divided into first dimension and second dimension sections. A first dimension section is where easyOCR will run, which are also the general sections of the car diagram (top, back, right, and left). The second dimension sections are the subparts of a first dimension section (doors, windows, etc).

After running the OCR on the first dimension section, the script will compare the position of the element found and the coordinates of the subparts to determine in which second dimension section the element can be found. This will add detail to the data extraction, which will allow for better decision-making in the future.

In [1]:
import cv2
import easyocr
import numpy as np
import pandas as pd

reader = easyocr.Reader(['pt', 'en'])

roi_general = pd.read_csv('./roi/roi_general.csv')

roi_right = pd.read_csv('./roi/roi_right.csv')
roi_left = pd.read_csv('./roi/roi_left.csv')
roi_top = pd.read_csv('./roi/roi_top.csv')
roi_back = pd.read_csv('./roi/roi_back.csv')


roi_general

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
/home/pedro/Desktop/psa-door-image-etl/.venv/lib/python3.12/site-packages/easyocr/detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open a

,roi_name,x,y,w,h
0,right,29,168,1137,439
1,left,1255,180,1145,374
2,top,48,850,1491,662
3,back,1567,605,856,492


In [17]:
import os
import re

extracted_data = pd.DataFrame(columns=['car_no', 'gen_roi', 'roi', 'flaw_type'])

def get_subpart_name(x, y, gen_roi):
    subroi_df = globals()[f'roi_{gen_roi}']

    matching_rows = subroi_df[
        (subroi_df['x'] <= x) & (x <= (subroi_df['x'] + subroi_df['w'])) &
        (subroi_df['y'] <= y) & (y <= (subroi_df['y'] + subroi_df['h']))
    ]

    return matching_rows['roi_name'].item()


def add_subpart_to_df(data, gen_roi, car_no):
    gen_roi_x = roi_general[roi_general['roi_name'] == gen_roi]['x'].item()
    gen_roi_y = roi_general[roi_general['roi_name'] == gen_roi]['y'].item()

    for flaw in data:
        x = flaw[0][0][0]
        y = flaw[0][0][1]

        absolute_x = x + gen_roi_x
        absolute_y = y + gen_roi_y
        
        roi = get_subpart_name(absolute_x, absolute_y, gen_roi)
        flaw_type = flaw[1]
        
        new_row = {'car_no': car_no, 'gen_roi': gen_roi, 'roi': roi, 'flaw_type': flaw_type}
        extracted_data.loc[len(extracted_data)] = new_row


def get_diagram_data(diagram_path):
    img = cv2.imread(diagram_path)
    car_no = re.search(r'[A-Z]{2}\d{4}[A-Z]{2}', diagram_path).group(0)

    for _, row in roi_general.iterrows():
        cutted_img = img[row['y']: row['y'] + row['h'], row['x']: row['x'] + row['w']]

        result = reader.readtext(cutted_img, text_threshold=0.1)

        add_subpart_to_df(result, row['roi_name'], car_no)


def iterate_diagrams(diagram_folder):
    for diagram in os.listdir(diagram_folder):
        get_diagram_data(diagram_folder + diagram)


iterate_diagrams('./diagrams/jpg_files/')
extracted_data

KeyboardInterrupt: 